In [136]:
import pandas as pd
import requests
import time

In [137]:
income_df = pd.read_csv('../data/income_data.csv')
income_df.head()

,year,geo_name,geo,income
0,2015,Census Tract 101.01,14000US04013010101,106818.0
1,2015,Census Tract 101.02,14000US04013010102,132822.0
2,2015,Census Tract 304.02,14000US04013030402,76069.0
3,2015,Census Tract 405.02,14000US04013040502,32424.0
4,2015,Census Tract 405.07,14000US04013040507,39721.0


In [138]:
final_new_df = pd.read_pickle('final_new_df.pkl')
final_new_df.head()

,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,price,rating,review_count
0,jersey-mikes-subs-gilbert-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,"[{'open': [{'is_overnight': False, 'start': '1...",sVHAdRWYLXlnsNw2TsdwxQ,True,"[891 E Baseline Rd, Ste 102, Gilbert, AZ 85233]",85233,Jersey Mike's Subs,$,3.0,29.0
1,sauce-pizza-and-wine-scottsdale-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,"[{'open': [{'is_overnight': False, 'start': '1...",4LB4xtFAwfvJRachr6Eaxw,False,"[14418 N Scottsdale Rd, Ste 181, Scottsdale, A...",85254,Sauce Pizza and Wine,$$,3.5,182.0
2,fuddruckers-phoenix-3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,NaN,hI_WID6sP7EOgPeFFJPPwQ,False,"[8941 North Black Canyon Hwy, Phoenix, AZ 85021]",85021,Fuddruckers,$$,3.5,99.0
3,peter-piper-pizza-scottsdale-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.464893,-111.916801,"[{'open': [{'is_overnight': False, 'start': '1...",PWifWVSyaaQpSgRimHviWQ,False,"[7607 E McDowell Rd, Ste 101, Scottsdale, AZ 8...",85257,Peter Piper Pizza,$$,3.0,21.0
4,sofias-mexican-food-gila-bend,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,"[{'open': [{'is_overnight': False, 'start': '0...",mkDoNt71KAKc6wb8M29qrA,False,"[530 W Pima St, Gila Bend, AZ 85337]",85337,Sofia's Mexican Food,$,4.0,97.0


In [139]:
def get_census_tract(x, y):
    URL = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={x}&y={y}&benchmark=4&vintage=4&format=json"
    for i in range(0, 5):    
        try:
            surl = URL.format(x=x, y=y);
            res = requests.get(surl).json()
            return res['result']['geographies']['Census Tracts'][0]['GEOID']
        except KeyError:
            print(surl)
        except ValueError:
            print(surl)
        time.sleep(1)
    return ''

In [141]:
def apply_census(data):
    return get_census_tract(data['coordinates.longitude'], data['coordinates.latitude'])

In [142]:
census_series = final_new_df.apply(apply_census, axis=1)

https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-112.199739&y=33.637163&benchmark=4&vintage=4&format=json
https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-111.980081281306&y=33.6749041548855&benchmark=4&vintage=4&format=json
https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-111.524043381214&y=32.9877155274153&benchmark=4&vintage=4&format=json
https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-111.977496972561&y=33.6108205644632&benchmark=4&vintage=4&format=json
https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-112.166366577148&y=33.3781433105469&benchmark=4&vintage=4&format=json
https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-112.358749389648&y=33.6448745727539&benchmark=4&vintage=4&format=json
https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-111.74184&y=33.307846&benchmark=4&vintage=4&format=json
https://geocoding.geo.census.gov/geocoder/geographies/

In [143]:
final_new_df['census_id'] = census_series

In [144]:
# final_new_df['income'] = 0

In [145]:
def get_income(census):
    census = '14000US' + census
    income = income_df.loc[income_df['geo'] == census]['income']
    if (len(income) > 0):
        try:
            return float(income.iloc[0])
        except ValueError:
            return float(0)
    else:
        return float(0)

In [146]:
final_new_df['income'] = final_new_df['census_id'].apply(get_income)

In [147]:
final_new_df

,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,price,rating,review_count,census_id,income
0,jersey-mikes-subs-gilbert-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",33.378695,-111.812692,"[{'open': [{'is_overnight': False, 'start': '1...",sVHAdRWYLXlnsNw2TsdwxQ,True,"[891 E Baseline Rd, Ste 102, Gilbert, AZ 85233]",85233,Jersey Mike's Subs,$,3.0,29.0,04013422307,53800.0
1,sauce-pizza-and-wine-scottsdale-2,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",33.617460,-111.926260,"[{'open': [{'is_overnight': False, 'start': '1...",4LB4xtFAwfvJRachr6Eaxw,False,"[14418 N Scottsdale Rd, Ste 181, Scottsdale, A...",85254,Sauce Pizza and Wine,$$,3.5,182.0,04013103220,84000.0
2,fuddruckers-phoenix-3,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.566990,-112.116241,NaN,hI_WID6sP7EOgPeFFJPPwQ,False,"[8941 North Black Canyon Hwy, Phoenix, AZ 85021]",85021,Fuddruckers,$$,3.5,99.0,04013105501,27835.0
3,peter-piper-pizza-scottsdale-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.464893,-111.916801,"[{'open': [{'is_overnight': False, 'start': '1...",PWifWVSyaaQpSgRimHviWQ,False,"[7607 E McDowell Rd, Ste 101, Scottsdale, AZ 8...",85257,Peter Piper Pizza,$$,3.0,21.0,04013218200,42807.0
4,sofias-mexican-food-gila-bend,"[{'alias': 'mexican', 'title': 'Mexican'}]",32.945603,-112.725909,"[{'open': [{'is_overnight': False, 'start': '0...",mkDoNt71KAKc6wb8M29qrA,False,"[530 W Pima St, Gila Bend, AZ 85337]",85337,Sofia's Mexican Food,$,4.0,97.0,04013723305,30764.0
5,five-guys-scottsdale,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",33.626620,-111.925025,"[{'open': [{'is_overnight': False, 'start': '1...",rxjHvIvV_BlzKa8ftRUpGA,False,"[15435 Scottsdale Rd, Scottsdale, AZ 85260]",85260,Five Guys,$,3.5,109.0,04013216816,103562.0
6,blimpie-phoenix-9,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",33.610515,-112.150589,"[{'open': [{'is_overnight': False, 'start': '1...",nrJHoyHClxK4zJp99VOshA,False,"[4267 W Thunderbird Rd, PHOENIX, AZ 85053]",85053,Blimpie,$,4.0,9.0,04013104212,60147.0
7,cornish-pasty-tempe-2,"[{'alias': 'british', 'title': 'British'}, {'a...",33.423059,-111.951843,"[{'open': [{'is_overnight': False, 'start': '1...",wl0QZqAzr1DelslQ02JGCQ,False,"[960 W University Dr, Tempe, AZ 85281]",85281,Cornish Pasty,$$,4.5,1550.0,04013318800,38564.0
8,chuck-e-cheeses-glendale,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",33.638055,-112.205433,"[{'open': [{'is_overnight': False, 'start': '1...",9eCMqkVa3E6yCiTmX21KqA,False,"[6773 W Bell Rd, Glendale, AZ 85308]",85308,Chuck E Cheese's,$$,2.5,48.0,04013071511,55625.0
9,sushi-ave-phoenix,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",33.639952,-112.019165,"[{'open': [{'is_overnight': False, 'start': '1...",6eCbK0Fg9N_DbdluVQx-pA,True,"[2945 E Bell Rd, Ste 102, Phoenix, AZ 85032]",85032,Sushi Ave,$$,4.0,148.0,04013103304,31023.0


In [148]:
final_new_df[final_new_df['income'] == 0.0]

,alias,categories,coordinates.latitude,coordinates.longitude,hours,id,is_closed,location.display_address,location.zip_code,name,price,rating,review_count,census_id,income
13,jody-maronis-sausage-kingdom-phoenix,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}]",33.434753,-112.006439,NaN,Q17zQBhfq9AKmvSmhDhjlA,False,"[3400 Sky Harbor Blvd, Sky Harbor Airport - Te...",85034,Jody Maroni's Sausage Kingdom,$$,2.0,23.0,04013113802,0.0
19,nathans-famous-hotdogs-phoenix,"[{'alias': 'hotdog', 'title': 'Hot Dogs'}]",33.434109,-111.996914,NaN,cQMhOnD_92IUwu9QaGIbrQ,False,"[3800 W Sky Harbor Blvd, T4/S3, Phoenix, AZ 85...",85034,Nathans Famous Hotdogs,$,2.0,12.0,04013113802,0.0
28,china-buffet-casa-grande,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",32.878520,-111.715780,NaN,ycM6ob_3okclY2Uh_SrZqw,True,"[1677 E Florence Blvd, Casa Grande, AZ 85222]",85222,China Buffet,$$,2.5,9.0,04021001407,0.0
32,grand-buffet-apache-junction,"[{'alias': 'chinese', 'title': 'Chinese'}]",33.416000,-111.572388,NaN,-isRTMRxqTNqYpE3SXcSTQ,True,"[2430 W Apache Trl, Apache Junction, AZ 85220]",85220,Grand Buffet,$,2.0,4.0,04021000308,0.0
36,romys-bbq-maricopa,"[{'alias': 'bbq', 'title': 'Barbeque'}]",33.052349,-112.040894,"[{'open': [{'is_overnight': False, 'start': '1...",rQ99zBn6LsBV-7EAaRDJ5w,True,"[44301 W Maricopa Casa Grande Hwy, Maricopa, A...",85239,Romy's BBQ,$$,5.0,5.0,04021001707,0.0
50,mcdonalds-maricopa,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",33.067138,-112.047711,"[{'open': [{'is_overnight': True, 'start': '00...",RBdVxPEH10Zi7SmZhUiVnw,False,"[20700 N John Wayne Pkwy, Maricopa, AZ 85239]",85239,McDonald's,$,2.0,25.0,04021001703,0.0
56,tommys-bistro-casa-grande,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",32.880050,-111.742520,"[{'open': [{'is_overnight': False, 'start': '1...",MzHWLt_fVbSamwNUSeJfwQ,True,"[913 E 8th St, Casa Grande, AZ 85222]",85222,Tommy's Bistro,$$,4.0,73.0,04021001403,0.0
63,los-favoritos-apache-junction,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",33.414743,-111.566262,"[{'open': [{'is_overnight': False, 'start': '0...",2QAD4Qf_L0tX7NygXMCEZw,False,"[1879 W Apache Trl, Apache Junction, AZ 85220]",85220,Los Favoritos,$,4.0,52.0,04021000312,0.0
136,silver-bullet-bar-casa-grande,"[{'alias': 'bars', 'title': 'Bars'}]",32.887047,-111.757607,NaN,pCzgK3W5kD3DCkSxL5CHEg,True,"[1401 N Pinal Avenue, Casa Grande, AZ 85222]",85222,Silver Bullet Bar,$$,3.0,3.0,04021001500,0.0
149,hong-kong-kitchen-casa-grande,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",32.879720,-111.749990,"[{'open': [{'is_overnight': False, 'start': '1...",iJQ5ueLlD8-LTdtygXsD9A,False,"[314 E Florence Blvd, Casa Grande, AZ 85222]",85222,Hong Kong Kitchen,$$,3.5,25.0,04021001403,0.0


In [150]:
final_new_df.to_pickle('final_new_df_income.pkl')